# Import File

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
df = pd.read_csv("household_power_consumption.csv")

# Cleaning Data

In [15]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Date                   1048575 non-null  object 
 1   Time                   1048575 non-null  object 
 2   Global_active_power    1048575 non-null  object 
 3   Global_reactive_power  1048575 non-null  object 
 4   Voltage                1048575 non-null  object 
 5   Global_intensity       1048575 non-null  object 
 6   Sub_metering_1         1048575 non-null  object 
 7   Sub_metering_2         1048575 non-null  object 
 8   Sub_metering_3         1044506 non-null  float64
dtypes: float64(1), object(8)
memory usage: 72.0+ MB


,Sub_metering_3
count,1.044506e+06
mean,5.933837e+00
std,8.210398e+00
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,1.700000e+01
max,3.100000e+01


In [16]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], 
                                dayfirst=True,  # karena format dd/mm/yyyy
                                errors='coerce')

In [17]:
df.set_index('datetime', inplace=True)

In [18]:
cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity',
        'Sub_metering_1','Sub_metering_2','Sub_metering_3']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# Visualisasi Data

In [ ]:
df.dropna(inplace=True)

df.head()

In [ ]:
df['Global_active_power'].resample('D').mean().plot(figsize=(12,5))
plt.title("Rata-rata Konsumsi Energi Harian")
plt.ylabel("kW")
plt.show()

In [ ]:
df['hour'] = df.index.hour
df.groupby('hour')['Global_active_power'].mean().plot(figsize=(10,5))
plt.title("Pola Konsumsi per Jam")
plt.xlabel("Jam")
plt.ylabel("kW")
plt.show()

In [ ]:
df[['Sub_metering_1','Sub_metering_2','Sub_metering_3']].mean().plot(kind='bar')
plt.title("Rata-rata Konsumsi Energi per Area Rumah")
plt.ylabel("Watt-hour")
plt.show()


# Analisis Korelasi Antar Variable

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(df[cols].corr(), annot=True, cmap="coolwarm")
plt.title("Matriks Korelasi Variabel Energi")
plt.show()

Berdasarkan analisis korelasi, konsumsi energi rumah tangga terutama dipengaruhi oleh besarnya arus listrik yang mengalir. Penggunaan energi tertinggi berasal dari pemanas air dan AC (Sub_metering_3), diikuti aktivitas dapur dan laundry. Selain itu, peningkatan beban konsumsi listrik menyebabkan penurunan tegangan. Temuan ini menunjukkan bahwa efisiensi energi dapat dicapai dengan mengatur jadwal penggunaan perangkat berdaya tinggi dan mengoptimalkan aktivitas konsumsi pada jam tertentu.

# Membuat Model Prediksi

In [ ]:
# Pilih fitur (X) dan target (y)
X = df[['Global_intensity', 'Voltage', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
y = df['Global_active_power']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R² Score:", r2)


In [ ]:
coeff_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
coeff_df

In [ ]:
# Buat DataFrame perbandingan aktual vs prediksi
result_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
})

# Tampilkan 10 data pertama
print("Perbandingan Nilai Aktual vs Prediksi:")
result_df.head(10)


In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel("Nilai Aktual")
plt.ylabel("Prediksi Model")
plt.title("Perbandingan Nilai Aktual vs Prediksi Konsumsi Energi")
plt.show()

Model regresi linear digunakan untuk memprediksi konsumsi energi berdasarkan arus listrik, tegangan, dan penggunaan daya pada beberapa area rumah. Model memberikan nilai RMSE sebesar 0.0452 dan R² sebesar 0.998 yang menunjukkan bahwa model mampu menjelaskan sebagian besar variasi konsumsi energi. Variabel dengan kontribusi terbesar terhadap konsumsi energi adalah Global_intensity dan Sub_metering_3, yang berarti pemanas air dan AC merupakan sumber konsumsi energi terbesar.